# Exomoon N-body Stability Map

blah blah


In [8]:
def calc_hill_radius(mass_primary, mass_secondary, sma_secondary):
    return sma_secondary * (mass_primary / (3 * mass_secondary))**(1./3)

HIP 41378 system (Santerne et al. 2019)

In [12]:
from astropy import constants

# HIP 41378
M_star = 1.16 * constants.M_sun.value
a_star = 1.37 * constants.au.value

# HIP 41378f
M_planet = 12 * constants.M_earth.value
R_planet = 9.2 * constants.R_earth.value

Make up a moon

In [36]:
import numpy as np

# hypothetical moon
M_moon = 0.0123 * M_planet   # Moon-Earth mass ratio
R_moon = ((3/(4*np.pi)) * M_moon / 5500 )**(1/3)   # Assumes desnity of Earth

print("R_moon: {:.3f} Earth radii \n".format(R_moon / constants.R_earth.value),
      "M_moon: {:.3f} Earth masses".format(M_moon / constants.M_earth.value))

R_moon: 0.528 Earth radii 
 M_moon: 0.148 Earth masses


In [42]:
hill_radius = calc_hill_radius(M_planet, M_star, a_star)   # mks
print("Hill radius: {:.3f} AU".format(hill_radius / constants.au.value))

Hill radius: 0.030 AU


In [40]:
print("Planet radius: {:.5f} AU".format(R_planet / constants.au.value))

Planet radius: 0.00039 AU


Our function looks like this:

```python

    import numpy as np
    import rebound
    import astropy.constants as constants


    def exomoon_simulation(params):
        """Run a hierarchical N-body simulation with one planet and one moon around the planet. 
        
        Args:
            params (tuple): eccentricity of planet and semi-major axis of moon (in fraction of Hill Radius)
        
        """
        
        # unpack params
        ecc_planet, a_moon = params
        
        # constants
        MEARTH_TO_MSUN = constants.M_earth.value / constants.M_sun.value
        DAY_TO_YEAR = 1. / 365.25
        MIN_DIST = 0.00039   # AU (planet radius)
        MAX_DIST = 0.030     # AU (Hill radius)
        
        # fixed params
        m_star = 1.16                                   # stellar mass
        mass_moon = 0.148 * MEARTH_TO_MSUN              # moon mass 
        mass_planet = 12. * MEARTH_TO_MSUN              # planet mass
        period_planet = 542.08 * DAY_TO_YEAR * 2*np.pi  # planet orbital period
        
        # hyperparams
        sim_time = 1000  # run for 1000 years
        N_outputs = 1000
        
        # random mean anomaly for moon
        ma_moon = np.random.uniform(0, 2*np.pi)
        
        # set up simulation
        sim = rebound.Simulation()
        sim.integrator = "ias15"
        
        sim.add(m=m_star, hash="star")  # add star
        sim.add(m=mass_planet, P=period_planet, e=ecc_planet, M=0., inc=np.pi/2, hash="planet")  # add planet around star
        sim.add(m=mass_moon, a=a_moon*MAX_DIST, e=0., M=ma_moon, inc=np.pi/2,
                primary=sim.particles[-1], hash="moon")  # add moon around planet
        
        sim.dt = 0.05 * sim.particles[-1].P  # first timestep is 5% of moon's orbital period
        sim.move_to_com()
        
        sim.exit_min_distance = MIN_DIST  # catch encounters

        # run simulation
        timescale = sim_time * (2 * np.pi)  # units where G=1
        ps = sim.particles  # an array of pointers that will update as the simulation runs
        times = np.linspace(0, timescale, N_outputs)

        try:
            for time in times:
                sim.integrate(time)
                dp_moon = ps["planet"] - ps["moon"]  # calculate the component-wise difference between moon and planet
                dist_sq_moon = dp_moon.x*dp_moon.x + dp_moon.y*dp_moon.y + dp_moon.z*dp_moon.z
                if dist_sq_moon > MAX_DIST**2:
                    return 0.  # moon escaped Hill sphere: unstable
                
        except rebound.Encounter:
            return 0.  # close encounter: unstable
        
        return 1.  # stable
```

In [1]:
from nbody_sim_functions import exomoon_simulation

In [2]:
exomoon_simulation((0.3, 0.4))

0.0

In [3]:
x_min, x_max = 0, 0.5      # planet eccentricity
y_min, y_max = 0.25, 0.55  # moon semi-major axis (in Hill radii)

In [5]:
import astroqtpy.quadtree as qt

In [6]:
nbody_tree = qt.NbodyQuadTree(x_min, x_max, y_min, y_max, exomoon_simulation,
                              split_threshold=0.2,
                              N_points=20,
                              N_proc=4,
                              verbose=True
                              )

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

cm = nbody_tree.draw_tree(ax, vmin=0, vmax=1, show_points)
plt.colorbar(cm, ax=ax, label='$f_{stab}$')

plt.show(); 